# 0. Импортирование библиотек

In [1]:
import pandas as pd
import json
import numpy as np
import sklearn as sk
import openpyxl
import time

from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils import shuffle

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

# 1. Методы бинарной классификации

### 1.1. Предобработка данных

In [2]:
def get_dataset(path):        
    """
    Преобразовать json-файлы с отзывами к единому датасету с двумя столбцами:
        1. сам отзыв
        2. класс отзыва - если рейтинг отзыва (по пятибальнной системе) сторого меньше 3,
                          то отзыву присваивается класс 0 (то есть отрицательный отзыв),
                          иначе - класс равен 1 (положительный отзыв)
    
    Входящие параметры
    -------------
    path : array
        Массив наименований файлов для импорта
    
    Возвращает
    -------------
    data : dataframe
        датасет вида отзыв - класс отзыва
    
    """
    
    data = pd.DataFrame()
    
    for i in range(len(path)):
        d0 = pd.read_json(path[i], lines=True)
        col = ['reviewText','overall']
        d0 = d0[col]                                # оставить в датафрейме только столбцы reviewText, overall
        d0.loc[d0['overall'] < 3, 'overall'] = 0    # пометить классом 0 отрицательные отзывы
        d0.loc[d0['overall'] >= 3, 'overall'] = 1   # пометить классом 1 положительные отзывы
        data = data.append(d0)
    
    return data

In [3]:
# Данные взяты по ссылке http://jmcauley.ucsd.edu/data/amazon/
# Представляют собой оцененные по пятибальной шкале отзывы товаров
# по категориям: Для красоты, Для детей, Здоровье и персональный уход

# массив наименований файлов данных
path = np.array(['reviews_Baby_5.json.gz',  
                 'reviews_Beauty_5.json.gz',
                 'reviews_Health_and_Personal_Care_5.json.gz'])

# кол-во записей каждого класса
N_1 = 15000    # кол-во положительных отзывов
N_2 = 15000    # кол-во отрицательных отзывов

# наилучшие параметры классификаторов
с_ = 0.28
n_ = 300
h_ = 7

In [4]:
#создаем dataframe для выгрузки количества отзывов по классам
reviews = pd.DataFrame(columns=['type_of_classification','name','count'])

reviews = reviews.append({'type_of_classification':'binary', 'name':'Negative', 'count':N_1}, ignore_index=True)
reviews = reviews.append({'type_of_classification':'binary', 'name':'Positive', 'count':N_2}, ignore_index=True)

In [5]:
full = get_dataset(path)         # получить датасет
full_prep = full.copy()

full_prep_1 = full_prep[full_prep['overall'] == 0][:N_1]     # взять N_1 отрицательных отзывов
full_prep_2 = full_prep[full_prep['overall'] == 1][:N_2]     # взять N_2 положительных отзывов
full_prep = pd.DataFrame()
full_prep = full_prep.append(full_prep_1).append(full_prep_2) # объединение выделенных отзывов

# разбиение данных на признаки и классы объектов
full2 = shuffle(full_prep, random_state=241)    # случайным образом перемешать упорядоченные данные
data = full2['reviewText']                      # выделение признаков объектов
target = full2['overall']                       # выделение классов объектов

# разбиение на обучающую и тестовую выборку
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3, random_state=42)

# Привести полученные данные к массиву numpy
X_train = X_train.values.astype('U')        # Преобразовать элементы массива к текстовым
X_test = X_test.values.astype('U')
y_train = y_train.values.astype('int32')    # Преобразовать элементы массива к числовым 
y_test = y_test.values.astype('int32')

# Векторизация признаков
# В параметре encoding выбрана кодировка
# в decode_error выбирается действие, если найден символ не из выбранной кодировки (replace значит, что символ заменяется)
# min_df значит, что игнорируются слова с меньшей частотой появления в отзывах
# если заполнен параметр stop_words, то убираются слова, котрые являются неинформативными (например, союзы, предлоги)
text = TfidfVectorizer(min_df=5, decode_error='replace', encoding='utf-8', stop_words='english')
X = text.fit_transform(X_train)

# наилучшие полученные результаты
classification_rates = pd.DataFrame(columns=['type_of_classification','category','method','parameter','value','acc','f1','roc'])

In [6]:
len(text.get_feature_names())

8209

In [7]:
#результат для облака слов
lsvc = LinearSVC(C = с_, random_state=42)
lsvc.fit(X, y_train)

feature_names = np.array(text.get_feature_names())
sorted_coef_index = np.argsort(lsvc.coef_[0])

print('Smallest Coefs: \n{}\n'.format(feature_names[sorted_coef_index[:10]]))
print('Largest Coefs: \n{}\n'.format(feature_names[sorted_coef_index[:-11:-1]]))

Smallest Coefs: 
['disappointed' 'returned' 'waste' 'useless' 'returning' 'idea'
 'disappointing' 'unfortunately' 'return' 'thought']

Largest Coefs: 
['perfect' 'great' 'easy' 'love' 'loves' 'complaint' 'perfectly' 'highly'
 'best' 'excellent']



In [8]:
#выгрузка облака слов в ексель
coefs = pd.DataFrame(columns=['type_of_classification','category_of_coef','word', 'coef'])

sorted_coef = lsvc.coef_[0]
Smallest_Coefs = sorted_coef[:10]
Largest_Coefs = sorted_coef[:-11:-1]

Smallest_Coefs_index = sorted_coef_index[:10]
Largest_Coefs_index = sorted_coef_index[:-11:-1]

for i in range(len(Largest_Coefs)):
    coefs = coefs.append({'type_of_classification': 'binary', 'category_of_coef': 'Largest_Coefs', 
                          'word': format(feature_names[Largest_Coefs_index[i]]),'coef': Largest_Coefs[i]}, ignore_index=True)
    
for i in range(len(Smallest_Coefs)):
    coefs = coefs.append({'type_of_classification': 'binary', 'category_of_coef': 'Smallest_Coefs',
                          'word': format(feature_names[Smallest_Coefs_index[i]]),'coef': Smallest_Coefs[i]}, ignore_index=True)

### 1.2.1. LinearSVC (Линейный метод опорных векторов)

In [9]:
def LSVC_clf(scores, c):
    """
    Оценка качества класификатора. Возвращает рассчитанные значения метрик в среднем для всей выборки
    и отдельно по классам Линейного классификатора опорных векторов
    
    Входящие параметры
    -------------
    scores : dataframe
        для записи полученных результатов
    
    c : float
        параметр l2-регуляризации классификатора
    
    Возвращает
    -------------
    scores : dataframe
        датафрейм с рассчитанными метриками
    
    """
    
    lsvc = LinearSVC(C = c, random_state=42)
    lsvc.fit(X, y_train)                               # проведение обучения на обучающей выборке
    lsvc_preds = lsvc.predict(text.transform(X_test))  # получение значения целевой переменной на тестовой выборке
    
    #метрики для всей выборки
    acc = accuracy_score(y_test, lsvc_preds)           # расчет метрики accuracy
    f1 = f1_score(y_test, lsvc_preds)                  # расчет метрики f-мера
    roc = roc_auc_score(y_test, lsvc_preds)            # расчет метрики roc_auc
    
    # запись полученных метрик
    scores = scores.append({'type_of_classification':'binary', 'category': 'summary', 'method': 'LSVC',
                            'parameter': 'c', 'value': c, 'acc': acc, 'f1': f1, 'roc': roc}, ignore_index=True)
    
    return scores

In [10]:
classification_rates = LSVC_clf(classification_rates, с_)

### 1.2.2. ExtraTreesClassifier (Классификатор экстремально рандомизированных деревьев)

In [11]:
def extree_clf(scores, n):
    """
    Оценка качества класификатора. Возвращает рассчитанные значения метрик в среднем для всей выборки
    и отдельно по классам Классификатора экстра-деревьев
    
    Входящие параметры
    -------------
    scores : dataframe
        для записи полученных результатов
    
    n : int
        количество деревьев  в лесу
    
    Возвращает
    -------------
    scores : dataframe
        датафрейм с рассчитанными метриками
    
    """
    # n_jobs=-1 означает, что выполняется параллельно на всех процессорах
    extree = ExtraTreesClassifier(n_estimators=n, random_state=42, n_jobs=-1)
    extree.fit(X, y_train)
    extree_preds = extree.predict(text.transform(X_test))
    
    #метрики для всей выборки
    acc = accuracy_score(y_test, extree_preds)
    f1 = f1_score(y_test, extree_preds)
    roc = roc_auc_score(y_test, extree_preds)
    
    # запись полученных метрик
    scores = scores.append({'type_of_classification':'binary', 'category': 'summary', 'method': 'ExTree',
                            'parameter': 'n', 'value': n, 'acc': acc, 'f1': f1, 'roc': roc}, ignore_index=True)
    
    return scores

In [12]:
classification_rates = extree_clf(classification_rates, n_)

### 1.2.3. MLPClassifier (Многослойный персептрон)

In [13]:
def mlp_clf(scores, h):
    """
    Оценка качества класификатора. Возвращает рассчитанные значения метрик в среднем для всей выборки
    и отдельно по классам классификатора Многослойный персептрон (алгоритма искусственной нейронной сети)
    
    Входящие параметры
    -------------
    scores : dataframe
        для записи полученных результатов
    
    h : int
        количество нейронов в скрытом слое
    
    Возвращает
    -------------
    scores : dataframe
        датафрейм с рассчитанными метриками
    
    """
    # alpha - параметр l2 регуляризации
    # max_iter - максимальное число итераций
    # activation='logistic' - логистическая сигмоидная функция активации скрытого слоя
    # early_stopping - автоматически откладывается 10% тренировочных данных в качестве валидации и завершается обучение,
    #                  когда оценка валидации не улучшается для последовательных эпох.
    mlp = MLPClassifier(hidden_layer_sizes=(h,), alpha=0.01, random_state=42, max_iter=1000, activation='logistic',
                       early_stopping=True)
    mlp.fit(X, y_train)
    mlp_pred = mlp.predict(text.transform(X_test))
    
    #метрики для всей выборки
    acc = accuracy_score(y_test, mlp_pred)
    f1 = f1_score(y_test, mlp_pred)
    roc = roc_auc_score(y_test, mlp_pred)
    
    # запись полученных метрик
    scores = scores.append({'type_of_classification':'binary', 'category': 'summary', 'method': 'MLP',
                            'parameter': 'h', 'value': h, 'acc': acc, 'f1': f1, 'roc': roc}, ignore_index=True)
    
    return scores

In [14]:
classification_rates = mlp_clf(classification_rates, h_)

In [15]:
classification_rates

,type_of_classification,category,method,parameter,value,acc,f1,roc
0,binary,summary,LSVC,c,0.28,0.861444,0.860405,0.861473
1,binary,summary,ExTree,n,300.00,0.850444,0.851697,0.850427
2,binary,summary,MLP,h,7.00,0.858667,0.857047,0.858707


### 1.3. Подбор оптимальных параметров

#### 1.3.1. Подбор значения параметра 'C'  для метода LSVM

In [16]:
lsvm_scores = pd.DataFrame(columns=['type_of_classification','category','method','parameter','value','acc','f1','roc'])

for c in np.arange(0.1, 0.31, 0.02):
    lsvm_scores = LSVC_clf(lsvm_scores, c)

print(lsvm_scores[['category','value','acc','f1','roc']])
print(lsvm_scores[lsvm_scores['category']=='summary'][['value','acc','f1','roc']])

   category  value       acc        f1       roc
0   summary   0.10  0.860111  0.858332  0.860155
1   summary   0.12  0.861556  0.860031  0.861594
2   summary   0.14  0.862111  0.860796  0.862146
3   summary   0.16  0.861444  0.860155  0.861478
4   summary   0.18  0.860556  0.859226  0.860590
5   summary   0.20  0.860889  0.859641  0.860922
6   summary   0.22  0.861222  0.859962  0.861255
7   summary   0.24  0.862333  0.861176  0.862365
8   summary   0.26  0.861667  0.860629  0.861695
9   summary   0.28  0.861444  0.860405  0.861473
10  summary   0.30  0.862222  0.861360  0.862247
    value       acc        f1       roc
0    0.10  0.860111  0.858332  0.860155
1    0.12  0.861556  0.860031  0.861594
2    0.14  0.862111  0.860796  0.862146
3    0.16  0.861444  0.860155  0.861478
4    0.18  0.860556  0.859226  0.860590
5    0.20  0.860889  0.859641  0.860922
6    0.22  0.861222  0.859962  0.861255
7    0.24  0.862333  0.861176  0.862365
8    0.26  0.861667  0.860629  0.861695
9    0.28  0

#### 1.3.2. Подбор значения параметра 'n'  для метода ExTree

In [17]:
extree_scores = pd.DataFrame(columns=['type_of_classification','category','method','parameter','value','acc','f1','roc'])

for n in range(150, 360, 50):
    extree_scores = extree_clf(extree_scores, n)

print(extree_scores[['category','value','acc','f1','roc']])

  category value       acc        f1       roc
0  summary   150  0.848333  0.849354  0.848321
1  summary   200  0.850889  0.851942  0.850875
2  summary   250  0.849889  0.851162  0.849871
3  summary   300  0.850444  0.851697  0.850427
4  summary   350  0.852333  0.853489  0.852318


#### 1.3.3. Подбор значения параметра 'h'  для метода MLP

In [18]:
mlp_scores = pd.DataFrame(columns=['type_of_classification','category','method','parameter','value','acc','f1','roc'])

for h in [5, 7, 10, 20, 50, 100]:
    mlp_scores = mlp_clf(mlp_scores, h)

print(mlp_scores[['category','value','acc','f1','roc']])

  category value       acc        f1       roc
0  summary     5  0.859889  0.858806  0.859918
1  summary     7  0.858667  0.857047  0.858707
2  summary    10  0.859111  0.857592  0.859149
3  summary    20  0.852778  0.850299  0.852833
4  summary    50  0.861333  0.858407  0.861400
5  summary   100  0.858111  0.855363  0.858173


# 2. Методы многоклассовой классификации

### 2.1. Предобработка данных

In [19]:
# функция чтения записей из файла
def read_file(path, i):      
    cl, p = path[i]
    
    rdata = pd.read_json(p, lines=True)
    row = pd.DataFrame({'class': [cl for j in range(len(rdata))]})   # создать столбец CLASS
    data = pd.concat([rdata, row], axis=1)                           # вставить столбец CLASS
    
    col = ['reviewText','class']
    return data[col]

# функция, формирующая полный датасет
def get_data(path):        
    data = pd.DataFrame()
    
    for i in range(len(path)):
        d0 = read_file(path,i)
        data = data.append(d0)
    
    return data

# вычисление AUC_ROC_SCORE для многоклассового случая
def multiclass_roc_auc_score(y_test, y_pred):
    lb = sk.preprocessing.LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(y_pred)
    return roc_auc_score(y_test, y_pred)

# вычисление AUC_ROC_SCORE для многоклассового случая для каждого класса
def multiclass_roc_auc_score_class(y_test, y_pred):
    lb = sk.preprocessing.LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(y_pred)
    return roc_auc_score(y_test, y_pred, average = None)

In [20]:
# название групп и путь к файлам

path = np.array(['baby','reviews_Baby_5.json.gz',  
                 'beauty','reviews_Beauty_5.json.gz',
                'health','reviews_Health_and_Personal_Care_5.json.gz']).reshape(-1,2)

# DATA FROM http://jmcauley.ucsd.edu/data/amazon/
# 5-star rattings groups: Beauty, Baby, Health and Personal Care
# I can't upload this files on GitHub, bcoz they're bigger than 25 MB


# кол-во записей каждой из групп товаров

N1 = 10000    # кол-во записей 1-й группы
N2 = 10000    # кол-во записей 2-й группы
N3 = 10000    # кол-во записей 3-й группы

# наилучшие параметры классификаторов
c_SVM = 1      # параметр для SVM
n_RF = 200       # параметр для RF
n_GB = 250       # параметр для Gboost
c_LG = 1       # параметр для LG
n_AB = 260     #параметр для AB

In [21]:
#создаем dataframe для выгрузки количества отзывов по классам
reviews = reviews.append({'type_of_classification':'multiclass', 'name':'Baby', 'count':N1}, ignore_index=True)
reviews = reviews.append({'type_of_classification':'multiclass', 'name':'Beauty', 'count':N2}, ignore_index=True)
reviews = reviews.append({'type_of_classification':'multiclass', 'name':'Health', 'count':N3}, ignore_index=True)

In [23]:
full = get_data(path)         # получить датасет
full_prep = full.copy()
full_prep['class'].replace({path[0][0]: 1, path[1][0]: 2, path[2][0]: 3}, inplace=True)    # заменить метки классов на числа
full_prep.head()

full_prep_1 = full_prep[full_prep['class'] == 1][:N1]     # взять N_1 записей из первой группы
full_prep_2 = full_prep[full_prep['class'] == 2][:N2]     # взять N_2 записей из второй группы
full_prep_3 = full_prep[full_prep['class'] == 3][:N3]     # взять N_3 записей из третей группы
full_prep = pd.DataFrame()
full_prep = full_prep.append(full_prep_1).append(full_prep_2).append(full_prep_3)


# предобработка. Выделение данных и целевой переменной
full2 = shuffle(full_prep, random_state=241)
data = full2['reviewText']
target = full2['class']
# предобработка. разбиение на ОБУЧЕНИЕ и ТЕСТ
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3, random_state=42)

X_train = X_train.values.astype('U')
X_test = X_test.values.astype('U')
y_train = y_train.values.astype('U')
y_test = y_test.values.astype('U')

# Векторизация признаков
# В параметре encoding выбрана кодировка
# в decode_error выбирается действие, если найден символ не из выбранной кодировки (replace значит, что символ заменяется)
# min_df значит, что игнорируются слова с меньшей частотой появления в отзывах
# если заполнен параметр stop_words, то убираются слова, котрые являются неинформативными (например, союзы, предлоги)
text = TfidfVectorizer(min_df=5, encoding='utf-8', decode_error='replace', stop_words='english')
X = text.fit_transform(X_train)

In [24]:
len(text.get_feature_names())

8978

### 2.2.1. LogisticRegression (Логистическая регрессия)

In [25]:
def LG(scores, c):
    lr = LogisticRegression(C = c, random_state = 241)
    lr.fit(X, y_train)
    lr_preds = lr.predict(text.transform(X_test))

    #метрики для всей выборки
    acc = accuracy_score(y_test, lr_preds)
    f1 = f1_score(y_test, lr_preds, average = 'macro')
    roc = multiclass_roc_auc_score(y_test, lr_preds)
    
    # запись полученных метрик
    scores = scores.append({'type_of_classification':'multiclass', 'category': 'summary', 'method': 'LR',
                            'parameter': 'c', 'value': c, 'acc': acc, 'f1': f1, 'roc': roc}, ignore_index=True)
    
    #метрики для каждого класса
    acc_class = confusion_matrix(y_test, lr_preds)
    acc_class = acc_class.diagonal()/acc_class.sum(axis=1)
    
    f1_class = f1_score(y_test, lr_preds, average = None)
    roc_class = multiclass_roc_auc_score_class(y_test, lr_preds)
    
    for i in range(len(f1_class)):
        scores = scores.append({'type_of_classification':'multiclass', 'category': i + 1, 'method': 'LR', 'parameter': 'c',
                                'value': c, 'acc': acc_class[i], 'f1': f1_class[i], 'roc': roc_class[i]}, ignore_index=True)
    return scores

In [26]:
classification_rates = LG(classification_rates, c_LG)

C:\Users\xxmotovp\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\xxmotovp\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


### 2.2.2. AdaBoostClassifier

In [28]:
def AB(scores, n):
    adaboost = AdaBoostClassifier(n_estimators = n, random_state = 241)
    adaboost.fit(X, y_train)
    adaboost_preds = adaboost.predict(text.transform(X_test))
    
    #метрики для всей выборки
    acc = accuracy_score(y_test, adaboost_preds)
    f1 = f1_score(y_test, adaboost_preds, average = 'macro')
    roc = multiclass_roc_auc_score(y_test, adaboost_preds)
    
    # запись полученных метрик
    scores = scores.append({'type_of_classification':'multiclass', 'category': 'summary', 'method': 'AB',
                            'parameter': 'n', 'value': n, 'acc': acc, 'f1': f1, 'roc': roc}, ignore_index=True)
    
    #метрики для каждого класса
    acc_class = confusion_matrix(y_test, adaboost_preds)
    acc_class = acc_class.diagonal()/acc_class.sum(axis=1)
    
    f1_class = f1_score(y_test, adaboost_preds, average = None)
    roc_class = multiclass_roc_auc_score_class(y_test, adaboost_preds)
    
    for i in range(len(f1_class)):
        scores = scores.append({'type_of_classification':'multiclass', 'category': i + 1, 'method': 'AB', 'parameter': 'n',
                                'value': n, 'acc': acc_class[i], 'f1': f1_class[i], 'roc': roc_class[i]}, ignore_index=True)
    return scores

In [29]:
classification_rates = AB(classification_rates, n_AB)

In [30]:
classification_rates

,type_of_classification,category,method,parameter,value,acc,f1,roc
0,binary,summary,LSVC,c,0.28,0.861444,0.860405,0.861473
1,binary,summary,ExTree,n,300.00,0.850444,0.851697,0.850427
2,binary,summary,MLP,h,7.00,0.858667,0.857047,0.858707
3,multiclass,summary,LR,c,1.00,0.914556,0.914354,0.935789
4,multiclass,1,LR,c,1.00,0.947815,0.953099,0.963291
5,multiclass,2,LR,c,1.00,0.899460,0.908038,0.929684
6,multiclass,3,LR,c,1.00,0.895286,0.881924,0.914393
7,multiclass,summary,AB,n,260.00,0.859111,0.859982,0.894290
8,multiclass,1,AB,n,260.00,0.885845,0.912175,0.928345
9,multiclass,2,AB,n,260.00,0.838731,0.859464,0.891616


### 2.3. Подбор оптимальных параметров для многоклассовой классификации

#### 2.3.1. Подбор значения параметра 'C'  для метода LR

In [31]:
lr_scores = pd.DataFrame(columns=['type_of_classification','category','method','parameter','value','acc','f1','roc'])

for C_ in np.arange(0.1, 1.05, 0.05):
    lr_scores = LG(lr_scores, C_)
    
print(lr_scores[['category','value','acc','f1','roc']])
print(lr_scores[lr_scores['category']=='summary'][['value','acc','f1','roc']])

C:\Users\xxmotovp\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\xxmotovp\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\xxmotovp\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\xxmotovp\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning

   category  value       acc        f1       roc
0   summary   0.10  0.892111  0.892116  0.918929
1         1   0.10  0.932485  0.941390  0.953688
2         2   0.10  0.857287  0.881526  0.907106
3         3   0.10  0.885185  0.853433  0.895992
4   summary   0.15  0.896778  0.896757  0.922440
5         1   0.15  0.934768  0.943539  0.955335
6         2   0.15  0.865722  0.887122  0.911738
7         3   0.15  0.888552  0.859609  0.900246
8   summary   0.20  0.900444  0.900372  0.925190
9         1   0.20  0.938030  0.945897  0.957303
10        2   0.20  0.872807  0.891301  0.915363
11        3   0.20  0.889226  0.863919  0.902905
12  summary   0.25  0.902667  0.902556  0.926856
13        1   0.25  0.939987  0.946936  0.958281
14        2   0.25  0.876518  0.893705  0.917384
15        3   0.25  0.890236  0.867027  0.904902
16  summary   0.30  0.904667  0.904533  0.928358
17        1   0.30  0.941292  0.947783  0.959018
18        2   0.30  0.880567  0.896137  0.919491
19        3   0.30  

#### 2.3.2. Подбор значения параметра 'n'  для метода AdaBoost

In [32]:
ada_scores = pd.DataFrame(columns=['type_of_classification','category','method','parameter','value','acc','f1','roc'])

for n_ in np.arange(10, 510, 50):
    ada_scores = AB(ada_scores, n_)
    
print(ada_scores[['category','value','acc','f1','roc']])
print(ada_scores[lr_scores['category']=='summary'][['value','acc','f1','roc']])

   category value       acc        f1       roc
0   summary    10  0.712444  0.718743  0.785116
1         1    10  0.622962  0.751969  0.802718
2         2    10  0.650135  0.730616  0.793258
3         3    10  0.867003  0.673643  0.759372
4   summary    60  0.828556  0.830669  0.871503
5         1    60  0.838552  0.885941  0.905204
6         2    60  0.803306  0.832081  0.870341
7         3    60  0.843434  0.773984  0.838964
8   summary   110  0.846667  0.847952  0.884974
9         1   110  0.872798  0.904971  0.921906
10        2   110  0.821188  0.846461  0.881353
11        3   110  0.845118  0.792423  0.851663
12  summary   160  0.855444  0.856628  0.891567
13        1   160  0.879322  0.911119  0.926516
14        2   160  0.835695  0.857242  0.889849
15        3   160  0.850505  0.801523  0.858337
16  summary   210  0.858222  0.859315  0.893669
17        1   210  0.877691  0.909275  0.925195
18        2   210  0.840418  0.861044  0.892790
19        3   210  0.855892  0.807625  0

C:\Users\xxmotovp\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


### Прошлогодние методы

In [33]:
def SVM(scores, c):
    svm = SVC(C = c, random_state=241, kernel='linear', decision_function_shape='ovo')
    svm.fit(X, y_train)
    svm_preds = svm.predict(text.transform(X_test))
    
    #метрики для всей выборки
    acc = accuracy_score(y_test, svm_preds)
    f1 = f1_score(y_test, svm_preds, average = 'macro')
    roc = multiclass_roc_auc_score(y_test, svm_preds)
    
    # запись полученных метрик
    scores = scores.append({'type_of_classification':'multiclass', 'category': 'summary', 'method': 'SVM',
                            'parameter': 'c', 'value': c, 'acc': acc, 'f1': f1, 'roc': roc}, ignore_index=True)
    
    #метрики для каждого класса
    acc_class = confusion_matrix(y_test, svm_preds)
    acc_class = acc_class.diagonal()/acc_class.sum(axis=1)
    
    f1_class = f1_score(y_test, svm_preds, average = None)
    roc_class = multiclass_roc_auc_score_class(y_test, svm_preds)
    
    for i in range(len(f1_class)):
        scores = scores.append({'type_of_classification':'multiclass', 'category': i + 1, 'method': 'SVM', 'parameter': 'c',
                                'value': c, 'acc': acc_class[i], 'f1': f1_class[i], 'roc': roc_class[i]}, ignore_index=True)
    return scores

In [34]:
classification_rates = SVM(classification_rates, c_SVM)

In [35]:
def RF(scores, n):    
    forest = RandomForestClassifier(n_estimators=n, random_state=241, n_jobs=-1) # n_jobs=-1 - parallel on all processors
    forest.fit(X, y_train)
    rf_preds = forest.predict(text.transform(X_test))
    
    #метрики для всей выборки
    acc = accuracy_score(y_test, rf_preds)
    f1 = f1_score(y_test, rf_preds, average = 'macro')
    roc = multiclass_roc_auc_score(y_test, rf_preds)
    
    # запись полученных метрик
    scores = scores.append({'type_of_classification':'multiclass', 'category': 'summary', 'method': 'RF',
                            'parameter': 'n', 'value': n, 'acc': acc, 'f1': f1, 'roc': roc}, ignore_index=True)

    #метрики для каждого класса
    acc_class = confusion_matrix(y_test, rf_preds)
    acc_class = acc_class.diagonal()/acc_class.sum(axis=1)
    
    f1_class = f1_score(y_test, rf_preds, average = None)
    roc_class = multiclass_roc_auc_score_class(y_test, rf_preds)
    
    for i in range(len(f1_class)):
        scores = scores.append({'type_of_classification':'multiclass', 'category': i + 1, 'method': 'RF', 'parameter': 'n',
                                'value': n, 'acc': acc_class[i], 'f1': f1_class[i], 'roc': roc_class[i]}, ignore_index=True)
    return scores

In [36]:
classification_rates = RF(classification_rates, n_RF)

In [37]:
def GB(scores, n): 
    gb = GradientBoostingClassifier(learning_rate=1, n_estimators=n, random_state=241)
    gb.fit(X, y_train)
    gb_preds = gb.predict(text.transform(X_test))
    
    #метрики для всей выборки
    acc = accuracy_score(y_test, gb_preds)
    f1 = f1_score(y_test, gb_preds, average = 'macro')
    roc = multiclass_roc_auc_score(y_test, gb_preds)
    
    # запись полученных метрик
    scores = scores.append({'type_of_classification':'multiclass', 'category': 'summary', 'method': 'GB',
                            'parameter': 'n', 'value': n, 'acc': acc, 'f1': f1, 'roc': roc}, ignore_index=True)
    
    #метрики для каждого класса
    acc_class = confusion_matrix(y_test, gb_preds)
    acc_class = acc_class.diagonal()/acc_class.sum(axis=1)
    
    f1_class = f1_score(y_test, gb_preds, average = None)
    roc_class = multiclass_roc_auc_score_class(y_test, gb_preds)
    
    for i in range(len(f1_class)):
        scores = scores.append({'type_of_classification':'multiclass', 'category': i + 1, 'method': 'GB', 'parameter': 'n',
                                'value': n, 'acc': acc_class[i], 'f1': f1_class[i], 'roc': roc_class[i]}, ignore_index=True)
    return scores

In [38]:
classification_rates = GB(classification_rates, n_GB)

In [39]:
classification_rates

,type_of_classification,category,method,parameter,value,acc,f1,roc
0,binary,summary,LSVC,c,0.28,0.861444,0.860405,0.861473
1,binary,summary,ExTree,n,300.00,0.850444,0.851697,0.850427
2,binary,summary,MLP,h,7.00,0.858667,0.857047,0.858707
3,multiclass,summary,LR,c,1.00,0.914556,0.914354,0.935789
4,multiclass,1,LR,c,1.00,0.947815,0.953099,0.963291
5,multiclass,2,LR,c,1.00,0.899460,0.908038,0.929684
6,multiclass,3,LR,c,1.00,0.895286,0.881924,0.914393
7,multiclass,summary,AB,n,260.00,0.859111,0.859982,0.894290
8,multiclass,1,AB,n,260.00,0.885845,0.912175,0.928345
9,multiclass,2,AB,n,260.00,0.838731,0.859464,0.891616


# 3. Загрузка результатов в структуру данных для дешборда

In [40]:
def export_excel(filename , datas):   
    # открываем excel-файл
    wb = openpyxl.load_workbook(filename)

    # очистить лист перед записью
    wb.remove(wb['results'])
    wb.create_sheet(title = 'results', index = 0)
    
    sheet = wb['results']

    field_names = ('type_of_classification', 'category', 'method', 'acc', 'f1', 'roc')

    for i, field in enumerate(field_names):
        cell = sheet.cell(row = 1, column = i+1)
        cell.value = field

    for row in range(2, len(datas)+2):
        for col, field in enumerate(field_names):
            cell = sheet.cell(row = row, column = col+1)
            cell.value = datas[field][row-2]

    wb.save(filename)

In [41]:
export_excel('dataset.xlsx', classification_rates)

In [42]:
def export_excel_2(filename, datas):
    # открываем excel-файл
    wb = openpyxl.load_workbook(filename)
    
    # очистить лист перед записью
    wb.remove(wb['dictionary'])
    wb.create_sheet(title = 'dictionary', index = 0)
    
    sheet = wb['dictionary']

    field_names = ['type_of_classification', 'method', 'method_full_name']

    for i, field in enumerate(field_names):
        cell = sheet.cell(row = 1, column = i+1)
        cell.value = field

        
    slovar = {'AB':'AdaBoost', 'ExTree':'ExtraTreesClassifier', 'LSVC':'LinearSupportVectorClassifier', 'MLP':'MLPClassifier', 
              'LR':'LogisticRegression', 'SVM':'SupportVectorClassifier', 'RF':'RandomForestClassifier', 
              'GB':'GradientBoostingClassifier'}
    usedCombinations = set()
    row_last = 1
    for row in range(2, len(datas)+2):
        combination = datas[field_names[0]][row - 2] + slovar[datas[field_names[1]][row-2]]
        if combination not in usedCombinations:
            
            row_last+=1
            
            cell = sheet.cell(row = row_last, column = 1)
            cell.value = datas[field_names[0]][row - 2]
            
            cell = sheet.cell(row = row_last, column = 2)
            cell.value = datas[field_names[1]][row-2]
            
            cell = sheet.cell(row = row_last, column = 3)
            cell.value = slovar[datas[field_names[1]][row-2]]
            
            usedCombinations.add(combination)
            
        else:
            combination = ''
    wb.save(filename)

In [43]:
export_excel_2('dataset.xlsx', classification_rates)

In [44]:
def export_excel_3(filename, X_train, X_test):
    # открываем excel-файл
    wb = openpyxl.load_workbook(filename)
    
    # очистить лист перед записью
    wb.remove(wb['data_description'])
    wb.create_sheet(title = 'data_description', index = 0)
    
    sheet = wb['data_description']

    field_names = ['sample', 'value']

    for i, field in enumerate(field_names):
        cell = sheet.cell(row = 1, column = i+1)
        cell.value = field
        
    sample = {'train':len(X_train), 'test':len(X_test)}
    
    cell = sheet.cell(row = 2, column = 1)
    cell.value = 'train'
    cell = sheet.cell(row = 2, column = 2)
    cell.value = sample['train']
    
    cell = sheet.cell(row = 3, column = 1)
    cell.value = 'test'
    cell = sheet.cell(row = 3, column = 2)
    cell.value = sample['test']
   
    wb.save(filename)

In [45]:
export_excel_3('dataset.xlsx', X_train, X_test)

In [46]:
def export_excel_4(filename , datas):   
    # открываем excel-файл
    wb = openpyxl.load_workbook(filename)

    # очистить лист перед записью
    wb.remove(wb['cloud_of_words'])
    wb.create_sheet(title = 'cloud_of_words', index = 0)
    
    sheet = wb['cloud_of_words']

    field_names = ('type_of_classification', 'category_of_coef', 'word', 'coef')

    for i, field in enumerate(field_names):
        cell = sheet.cell(row = 1, column = i+1)
        cell.value = field

    for row in range(2, len(datas)+2):
        for col, field in enumerate(field_names):
            cell = sheet.cell(row = row, column = col+1)
            cell.value = datas[field][row-2]

    wb.save(filename)

In [47]:
export_excel_4('dataset.xlsx', coefs)

In [48]:
def export_excel_5(filename , datas):   
    # открываем excel-файл
    wb = openpyxl.load_workbook(filename)

    # очистить лист перед записью
    wb.remove(wb['reviews'])
    wb.create_sheet(title = 'reviews', index = 0)
    
    sheet = wb['reviews']

    field_names = ('type_of_classification', 'name', 'count')

    for i, field in enumerate(field_names):
        cell = sheet.cell(row = 1, column = i+1)
        cell.value = field

    for row in range(2, len(datas)+2):
        for col, field in enumerate(field_names):
            cell = sheet.cell(row = row, column = col+1)
            cell.value = datas[field][row-2]

    wb.save(filename)

In [49]:
export_excel_5('dataset.xlsx', reviews)